# Translate between `OptimizationProblem` and Docplex

If you are used to the Docplex workflow, you can seamlessly translate between a Docplex model and an `OptimizationProblem`  with the `from_docplex_mp` and `to_docplex_mp` translators. This translation allows to leverage Docplex capabilities such as **LP file loading** without too many additional steps.


<div style="background-color:#fff3cd; color:#856404; padding:15px; border:1px solid #ffeeba; border-radius:5px; font-size:16px;">
<strong>Note:</strong> Docplex does not support higher-order terms in its formulation. This conversion is limited to constant, linear, and quadratic terms.
</div>

In [1]:
from docplex.mp.model import Model

docplex_mod = Model(name="docplex model")
x = docplex_mod.binary_var(name="x")
y = docplex_mod.integer_var(name="y", lb=-1, ub=5)
z = docplex_mod.continuous_var(name="z", lb=-1, ub=5)

docplex_mod.minimize(3 + x + 2 * x * y - z * z)
docplex_mod.add_constraint(x + 2 * y == 3, "lin_eq")
docplex_mod.add_constraint(x + 2 * y <= 3, "lin_leq")
docplex_mod.add_constraint(x + 2 * y >= 3, "lin_geq")

print(docplex_mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex model

Minimize
 obj: x + [ 4 x*y - 2 z^2 ]/2 + 3
Subject To
 lin_eq: x + 2 y = 3
 lin_leq: x + 2 y <= 3
 lin_geq: x + 2 y >= 3

Bounds
 0 <= x <= 1
 -1 <= y <= 5
 -1 <= z <= 5

Binaries
 x

Generals
 y
End



When you display your problem as LP format using `export_as_lp_string`,
`Binaries` denotes binary variables and `Generals` denotes integer variables.

If variables are not included in either `Binaries` or `Generals`, such variables are continuous ones with default lower bound == 0 and upper bound == infinity.

Note that you cannot use `'e`' or `'E'` as the first character of names due to the [specification of LP format](https://www.ibm.com/docs/en/icos/22.1.0?topic=representation-variable-names-in-lp-file-format).

The following snippet shows the `from_docplex_mp` translator:

In [2]:
from qiskit_addon_opt_mapper.translators import from_docplex_mp

mod2 = from_docplex_mp(docplex_mod)
print(mod2.prettyprint())

Problem name: docplex model

Minimize
  2*x*y - z^2 + x + 3

Subject to
  Linear constraints (3)
    x + 2*y == 3  'lin_eq'
    x + 2*y <= 3  'lin_leq'
    x + 2*y >= 3  'lin_geq'

  Integer variables (1)
    -1 <= y <= 5

  Continuous variables (1)
    -1 <= z <= 5

  Binary variables (1)
    x



As stated before, you can also convert a `OptimizationProblem` to a Docplex model using the `to_docplex` function.

In [3]:
from qiskit_addon_opt_mapper.translators import to_docplex_mp

docplex_mod2 = to_docplex_mp(mod2)
print(docplex_mod2.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex model

Minimize
 obj: x + [ 4 x*y - 2 z^2 ]/2 + 3
Subject To
 lin_eq: x + 2 y = 3
 lin_leq: x + 2 y <= 3
 lin_geq: x + 2 y >= 3

Bounds
 0 <= x <= 1
 -1 <= y <= 5
 -1 <= z <= 5

Binaries
 x

Generals
 y
End



# Load LP files using Docplex

Once you are used to the Docplex translators, loading from LP files becomes very simple. You can use the `docplex.mp.ModelReader` class followed by a translation step:

In [ ]:
from docplex.mp.model_reader import ModelReader

model = ModelReader.read("data/my_lp_file.lp")

op3 = from_docplex_mp(model)
print(op3.prettyprint())

Problem name: my_lp_file

Minimize
  2*x0^2 + 0.5*x0*x1 + 0.5*x1^2 + 2*x0 + 2*x1

Subject to
  Linear constraints (1)
    x0 + x1 == 2  'constraint'

  Integer variables (2)
    0 <= x0 <= 4
    0 <= x1 <= 4

